# Sprint 機械学習スクラッチ入門

## 1.このSprintについて

Sprintの目的
機械学習スクラッチの準備をする

どのように学ぶか
今後の機械学習スクラッチ課題で作成するモデルを、scikit-learnを用いて一度動かしておきます。これまでの復習を兼ねたスクラッチ課題の準備です。


## 2.スクラッチ

このSprintでは機械学習手法のスクラッチ課題に取り組む準備を行います。scikit-learnを用いて分類・回帰問題を解くコードを書いておき、今後のSprintではそれと同じ動作をするクラスをスクラッチで作成していきます。

スクラッチの意義

ここでのスクラッチとは、NumPyなどの基本的なライブラリを組み合わせることで、scikit-learnのような応用的なライブラリと同じ機能のクラス・関数を自作することを指します。

スクラッチをすることでscikit-learnなどのライブラリを動かすだけでは掴みづらい、アルゴリズムの深い理解を目指します。コーディングのスキル向上も兼ねますが、それは主な目的ではありません。

以下のような効果を狙っています。

- 新たな手法に出会った時に理論・数式を理解しやすくする
- ライブラリを使う上での曖昧さを減らす
- 既存の実装を読みやすくする

## 【問題1】train_test_splitのスクラッチ
スクラッチの練習として、scikit-learnのtrain_test_splitを自作してみます。以下の雛形をベースとして関数を完成させてください。

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html" style="text-decoration:none">
    sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation
</a>

なお、作成した関数がscikit-learnのtrain_test_splitと同じ動作をしているか必ず確認をするようにしましょう。

**雛形**

In [1]:
import numpy as np
import pandas as pd

In [2]:
def scratch_train_test_split(X, y, train_size=0.8,):
    """
    検証データを分割する。

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, )
      正解値
    train_size : float (0<train_size<1)
      何割をtrainとするか指定

    Returns
    ----------
    X_train : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    X_test : 次の形のndarray, shape (n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape (n_samples, )
      訓練データの正解値
    y_test : 次の形のndarray, shape (n_samples, )
      検証データの正解値
    """
    
    n_X = len(X)
    n_y = len(y)
    assert n_X == 0 or n_X is not None, "[ERROR] The value of X cannot be processed properly."
    assert n_y == 0 or n_y is not None, "[ERROR] The value of y cannot be processed properly."
    assert n_X == n_y, "[ERROR] X and y have different number of rows."
    
    test_size = 1 - train_size

    X_test = X.sample(frac=test_size, random_state=0).sort_index()
    X_train = X.drop(X_test.index, axis=0)

    y_test = y.drop(X_train.index, axis=0)
    y_train = y.drop(X_test.index, axis=0)

    return X_train, X_test, y_train, y_test

In [3]:
# サンプルデータ用意
a = np.arange(1, 11).reshape(10, 1)
b = np.arange(11, 21).reshape(10, 1)
ab = np.hstack([a,b])
df = pd.DataFrame(ab, columns=['a', 'b'])
ser = pd.Series(np.zeros(10))
display(df)
display(ser)

,a,b
0,1,11
1,2,12
2,3,13
3,4,14
4,5,15
5,6,16
6,7,17
7,8,18
8,9,19
9,10,20


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
dtype: float64

In [4]:
# スクラッチ関数実行
a,b,c,d = scratch_train_test_split(df, ser)
display(a)
display(b)
display(c)
display(d)

,a,b
0,1,11
1,2,12
3,4,14
4,5,15
5,6,16
6,7,17
7,8,18
9,10,20


,a,b
2,3,13
8,9,19


0    0.0
1    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
9    0.0
dtype: float64

2    0.0
8    0.0
dtype: float64

### sklearnのtrain_test_splitと比較
同じ結果になった。

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df, ser, test_size=0.2, random_state=0)

In [7]:
display(x_train.sort_index())
display(x_test)
display(y_train.sort_index())
display(y_test)

,a,b
0,1,11
1,2,12
3,4,14
4,5,15
5,6,16
6,7,17
7,8,18
9,10,20


,a,b
2,3,13
8,9,19


0    0.0
1    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
9    0.0
dtype: float64

2    0.0
8    0.0
dtype: float64

## 3.scikit-learnを用いて機械学習を行うコードを作成


scikit-learnを使ったコードを作成していきます。

検証データの分割には問題1で作成した自作の関数を用いてください。クロスバリデーションではなくホールドアウト法で構いません。

## 分類問題
分類は3種類の手法をスクラッチします。

- ロジスティック回帰
- SVM
- 決定木

ロジスティック回帰はscikit-learnにおいてLogisticRegressionクラスとSGDClassifierクラスの2種類から使用できます。ここでは勾配降下法を用いて計算するSGDClassifierクラスを利用してください。引数でloss="log"とすることでロジスティック回帰の計算になります。

- <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier" style="text-decoration:none">sklearn.linear_model.SGDClassifier — scikit-learn 0.21.3 documentation<a>
- <a href="https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC" style="text-decoration:none">sklearn.svm.SVC — scikit-learn 0.21.3 documentation</a>
- <a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier" style="text-decoration:none">sklearn.tree.DecisionTreeClassifier — scikit-learn 0.21.3 documentation</a>

データセットは3種類用意します。

1つ目は事前学習期間同様にirisデータセットです。

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html" style="text-decoration:none">sklearn.datasets.load_iris — scikit-learn 0.20.2 documentation</a>

2値分類としたいため、以下の2つの目的変数のみ利用します。特徴量は4種類全て使います。

- virgicolorとvirginica

残り2つは特徴量が2つのデータセットを人工的に用意します。以下のコードで説明変数X,目的変数yが作成可能です。「シンプルデータセット1」「シンプルデータセット2」とします。特徴量が2つであるため可視化が容易です。

シンプルデータセット1作成コード

In [8]:
import numpy as np
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X1 = X[random_index]
y1 = y[random_index]

シンプルデータセット2作成コード

In [9]:
X2 = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
       [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
       [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
       [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
       [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
       [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
       [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
       [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
       [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
       [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
       [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
       [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
       [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
       [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
       [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
       [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
       [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
       [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
       [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
       [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y2 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## 【問題2】 分類問題を解くコードの作成
上記3種類の手法で3種類のデータセットを学習・推定するコードを作成してください。

## データ1

### データ用意

In [10]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.datasets import load_iris

In [11]:
iris_dataset = load_iris()
columns_list = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
df = pd.DataFrame(iris_dataset['data'], columns=columns_list)
df['target'] = iris_dataset['target']
df = df[df['target'] != 0]
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


### ロジスティック回帰

In [12]:
# データ分割
X_train, X_test, y_train, y_test = scratch_train_test_split(df.drop('target', axis=1), df.target)

# インスタンス生成・モデル学習
model01_1 = SGDClassifier(loss='log')
model01_1.fit(X_train, y_train)
print(model01_1)

# 予測
y_pred01_1 = model01_1.predict(X_test)
print(y_pred01_1)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred01_1)))

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
[1 1 1 1 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2]
正解率：0.9


### サポートベクターマシン

In [13]:
from sklearn.svm import SVC

In [14]:
# インスタンス生成・学習
model02_1 = SVC()
model02_1.fit(X_train, y_train)
print(model02_1)

# 予測
y_pred02_1 = model02_1.predict(X_test)
print(y_pred02_1)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred02_1)))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[1 1 1 1 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2]
正解率：0.9


### 決定木

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
# インスタンス生成・学習
model03_1 = DecisionTreeClassifier()
model03_1.fit(X_train, y_train)
print(model03_1)

# 予測
y_pred03_1 = model03_1.predict(X_test)
print(y_pred03_1)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred03_1)))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
[1 2 1 1 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2]
正解率：0.85


## データ2

### データ用意

In [17]:
df_X1 = pd.DataFrame(X1)
df_X1.head()

,0,1
0,0.772383,-2.291673
1,-0.593349,1.667883
2,-2.076486,0.487468
3,0.119227,3.625380
4,-3.130006,-0.156732


In [18]:
df_y1 = pd.DataFrame(y1)
df_y1.head()

,0
0,-1
1,1
2,1
3,1
4,1


### ロジスティック回帰

In [19]:
# データ分割
X_train, X_test, y_train, y_test = scratch_train_test_split(df_X1, df_y1)

# インスタンス生成・学習
model01_2 = SGDClassifier(loss='log')
model01_2.fit(X_train, y_train)
print(model01_2)

# 予測
y_pred01_2 = model01_2.predict(X_test)
print(y_pred01_2)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred01_2)))

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
[ 1 -1  1  1  1  1 -1  1  1  1 -1  1  1 -1  1 -1  1 -1  1  1  1  1  1  1
  1  1  1  1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1 -1  1 -1  1  1 -1  1  1  1
 -1  1  1  1 -1  1  1 -1 -1  1  1  1  1 -1  1 -1  1  1  1  1  1 -1  1 -1
  1 -1 -1 -1  1  1  1 -1  1 -1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1
  1 -1  1 -1]
正解率：1.0


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### サポートベクターマシン

In [20]:
# インスタンス生成・学習
model02_2 = SVC()
model02_2.fit(X_train, y_train)
print(model02_2)

# 予測
y_pred02_2 = model02_2.predict(X_test)
print(y_pred02_2)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred02_2)))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[ 1 -1  1  1  1  1 -1  1  1  1 -1  1  1 -1  1 -1  1 -1  1  1  1  1  1  1
  1  1  1  1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1 -1  1 -1  1  1 -1  1  1  1
 -1  1  1  1 -1  1  1 -1 -1  1  1  1  1 -1  1 -1  1  1  1  1  1 -1  1 -1
  1 -1 -1 -1  1  1  1 -1  1 -1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1
  1 -1  1 -1]
正解率：1.0


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 決定木

In [21]:
# インスタンス生成・学習
model03_2 = DecisionTreeClassifier()
model03_2.fit(X_train, y_train)
print(model03_2)

# 予測
y_pred03_2 = model03_2.predict(X_test)
print(y_pred03_2)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred03_2)))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
[ 1 -1  1  1  1  1 -1  1  1  1 -1  1  1 -1  1 -1  1  1  1  1  1  1  1  1
  1  1  1  1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1 -1  1 -1  1  1 -1  1  1  1
 -1  1  1  1 -1  1  1 -1 -1  1  1  1  1 -1  1 -1  1  1  1  1  1 -1  1 -1
  1 -1 -1 -1  1  1  1 -1  1 -1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1 -1  1
  1 -1  1 -1]
正解率：0.99


## データ3

### データ用意

In [22]:
df_X2 = pd.DataFrame(X2)
df_X2.head()

,0,1
0,-0.44699,-2.8073
1,-1.46210,-2.4586
2,0.10645,1.9242
3,-3.59440,-4.0112
4,-0.98880,4.5718


In [23]:
df_y2 = pd.DataFrame(y2)
df_y2.head()

,0
0,0
1,0
2,0
3,0
4,0


### ロジスティック回帰

In [24]:
# データ分割
X_train, X_test, y_train, y_test = scratch_train_test_split(df_X2, df_y2)

# インスタンス生成・学習
model01_3 = SGDClassifier(loss='log')
model01_3.fit(X_train, y_train)
print(model01_3)

# 予測
y_pred01_3 = model01_3.predict(X_test)
print(y_pred01_3)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred01_3)))

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
[1 0 1 1 0 1 1 0]
正解率：0.375


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### サポートベクターマシン

In [25]:
# インスタンス生成・学習
model02_3 = SVC()
model02_3.fit(X_train, y_train)
print(model02_3)

# 予測
y_pred02_3 = model02_3.predict(X_test)
print(y_pred02_3)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred02_3)))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[1 0 0 1 0 1 0 0]
正解率：0.375


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 決定木

In [26]:
# インスタンス生成・学習
model03_3 = DecisionTreeClassifier()
model03_3.fit(X_train, y_train)
print(model03_3)

# 予測
y_pred03_3 = model03_3.predict(X_test)

# score
print('正解率：{}'.format(metrics.accuracy_score(y_test, y_pred03_3)))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
正解率：0.75


## 回帰問題
回帰は1種類をスクラッチします。

- 線形回帰

線形回帰は勾配降下法を用いて計算するSGDRegressorクラスを利用してください。

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html" style="text-decoration: none">
    sklearn.linear_model.SGDRegressor — scikit-learn 0.21.3 documentation
</a>

データセットは事前学習期間同様にHouse Pricesコンペティションのものを使います。

<a href="https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data" style="text-decoration: none">House Prices: Advanced Regression Techniques    
</a>


train.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


### データ用意

In [27]:
df3 = pd.read_csv('../../data/house-prices-advanced-regression-techniques/train.csv')
df3.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [28]:
df_X3 = df3.loc[:, ['GrLivArea', 'YearBuilt']]
df_y3 = df3.loc[:, ['SalePrice']]

In [29]:
df_X3.head()

,GrLivArea,YearBuilt
0,1710,2003
1,1262,1976
2,1786,2001
3,1717,1915
4,2198,2000


In [30]:
df_y3.head()

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


## 【問題3】 回帰問題を解くコードの作成
線形回帰でHouse Pricesデータセットを学習・推定するコードを作成してください。

In [31]:
from sklearn.linear_model import SGDRegressor

In [32]:
# データ分割
X_train, X_test, y_train, y_test = scratch_train_test_split(df_X3, df_y3)

# インスタンス生成・学習
model04 = SGDRegressor()
model04.fit(X_train, y_train)
print(model04)

# 予測
y_pred04 = model04.predict(X_test)
# print(y_pred04)

# score
print('決定係数R^2 : {}'.format(model04.score(X_test, y_test)))
print('MAE(平均二乗誤差) : {}'.format(metrics.mean_squared_error(y_test, y_pred04)))

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)
決定係数R^2 : -6.157542983395936e+20
MAE(平均二乗誤差) : 4.252306799023522e+30


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
